In [4]:
import pandas as pd
import numpy as np
import requests
import zipfile
import os
import time

from IPython.display import clear_output
from scipy.stats import percentileofscore
from scipy.io import mmread

In [ ]:
project = {
    'project_id': 'p1',
    'genes': [
        {
            'name': 'gene_name1',
            'cell_types': [
                {'name': 'cell_type1', 'percentil': 81.9},
                {'name': 'cell_type2', 'percentil': 0.0}
            ]
        },
        {
            'name': 'gene_name2',
            'cell_types': [
                {'name': 'cell_type1', 'percentil': 56.9},
                {'name': 'cell_type2', 'percentil': 98.0}
            ]
        }
    ]
}

In [45]:
def get_percentiles_from_project(project_ID):
    
    # Check if percentil already exists
    if os.path.exists(f'../SingleCell-Files/percentiles/{project_ID}.percentiles.csv'):
        return None, None
    
    print(f"Reading files for {project_ID}...")
    # Read project matrix and metadata
    matrix, metadata, cell_names, gen_names = read_files(project_ID)
    clear_output(wait=True)
    
    if matrix is None:
        remove_project_files(project_ID)
        return None, None
    
    print(f"Getting cell type groups for {project_ID}...")
    # Generate cell type groups with metadata
    cell_type_groups = get_cell_type_groups(metadata, cell_names)
    clear_output(wait=True)

    print(f"Generating percentiles for {project_ID}...")
    # Get mean percentiles of each cell type
    get_percentiles(matrix, cell_type_groups)
    clear_output(wait=True)

    print(f"Percentiles created for {project_ID}!")
    
    # Remove project files, we don't need them anymore
    remove_project_files(project_ID)
    
    return cell_type_groups, gen_names['Gen_Name'].tolist()

In [46]:
def read_files(project_ID):
    # Download matrix using ontology link
    download_matrix(project_ID)
    
    # Read matrix
    matrix_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx'
    matrix = mmread(matrix_file_name).transpose()
    
    # Read metadata
    metadata = pd.read_csv(f'https://www.ebi.ac.uk/gxa/sc/experiment/{project_ID}/download?fileType=experiment-design&accessKey=', sep='\t')
    
    if 'Sample Characteristic[cell type]' not in metadata.columns:
        return None, None, None, None
    
    metadata = metadata[['Assay', 'Sample Characteristic[cell type]']]
    
    # Read cell names
    cells_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx_cols'
    cell_names = pd.read_csv(cells_file_name, header=None, names=['Assay'])
    
    # Read gen names
    gens_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx_rows'
    gen_names = pd.read_csv(gens_file_name, header=None, names=['Gen_Name'])
    
    gen_names['Gen_Name'] = gen_names['Gen_Name'].apply(lambda x: x.split('\t')[1])
    
    return matrix, metadata, cell_names, gen_names

In [47]:
def download_matrix(project_ID):
    
    server_name = 'http://194.4.103.244:3030'
    service_name = 'ds'
    request_url = server_name + '/' + service_name
    
    path_to_links = '../SingleCell-Files/downloads/'
    
    query = '''
        PREFIX a: <http://www.semanticweb.org/alicia/ontologies/2020/8/singleCellRepositories#> 
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT 
            ?normalisedCountsLink ?experimentDesignLink
        WHERE
        {
        '''\
        + \
        f'''
            ?projectID rdf:type a:Project ;
                       a:PR.hasProjectID "{project_ID}" ;
                       a:SPR.hasNormalisedCountsLink ?normalisedCountsLink ;
                       a:SPR.hasExperimentDesignLink ?experimentDesignLink .
        '''\
        +\
        '''      
        }
    '''
    response = requests.post(request_url,
       data={'query': query})
    
    if not response.json()['results']['bindings']:
        return None
    
    normalisedCountsLink = response.json()['results']['bindings'][0]['normalisedCountsLink']['value'] 
    
    # download the file contents in binary format
    response = requests.get(normalisedCountsLink)
    
    zip_name = path_to_links + project_ID + ".zip"
    
    # open method to open a file on your system and write the contents
    with open(zip_name, "wb") as code:
        code.write(response.content)
        
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extract(project_ID + '.aggregated_filtered_normalised_counts.mtx', path=path_to_links)
        zip_ref.extract(project_ID + '.aggregated_filtered_normalised_counts.mtx_rows', path=path_to_links)
        zip_ref.extract(project_ID + '.aggregated_filtered_normalised_counts.mtx_cols', path=path_to_links)
        
    os.remove(zip_name)

In [48]:
def get_cell_type_groups(metadata, cell_names):
    # Merge both dataframes so we get the cells of the matrix with their corresponding type
    cell_types = pd.merge(
        cell_names,
        metadata,
        how="inner",
        on='Assay'
    )
    
    # Group by cell type
    grouped = cell_types.groupby(by='Sample Characteristic[cell type]')

    groups = []

    # For each group, assign its name and the index of the matrix for this type
    for name, group in grouped:
        groups.append({
            'name': name,
            'index': group.index
        })

    return groups

In [49]:
def get_percentiles(matrix, cell_type_groups):
    for group in cell_type_groups:

        print(group['name'])
        submatrix = matrix.A[group['index']]
        print(submatrix.shape)
        mean = np.mean(submatrix, axis=0)
                
        group['percentiles'] = [percentileofscore(mean, x, 'strict') for x in mean]

In [50]:
def remove_project_files(project_ID):
    matrix_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx'
    cells_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx_cols'
    gens_file_name = f'../SingleCell-Files/downloads/{project_ID}.aggregated_filtered_normalised_counts.mtx_rows'

    os.remove(matrix_file_name)
    os.remove(cells_file_name)
    os.remove(gens_file_name)

# Test percentil creation with a project

In [17]:
%%time
# groups, gen_names = get_percentiles_from_project('E-GEOD-100911')
groups, gen_names = get_percentiles_from_project('E-MTAB-6386')
# groups, gen_names = get_percentiles_from_project('E-GEOD-139324')

Percentiles created!
CPU times: user 1.08 s, sys: 53.3 ms, total: 1.14 s
Wall time: 4.24 s


In [81]:
dic = {}
for d in [{'gen_name': gen_names}] + [{group['name']: group['percentiles']} for group in groups]:
    dic.update(d)

In [82]:
df = pd.DataFrame(dic)
df

,gen_name,leukocyte
0,ENSG00000000003,35.400328
1,ENSG00000000419,87.615272
2,ENSG00000000457,62.690683
3,ENSG00000000460,52.667414
4,ENSG00000000938,90.881669
...,...,...
23201,ENSG00000288529,4.369560
23202,ENSG00000288534,47.698871
23203,ENSG00000288550,55.494269
23204,ENSG00000288558,65.164182


In [83]:
# df.to_csv('../SingleCell-Files/E-GEOD-100911.percentiles.csv', index=False)
# df.to_csv('../SingleCell-Files/E-MTAB-6386.percentiles.csv', index=False)
df.to_csv('../SingleCell-Files/E-GEOD-139324.percentiles.csv', index=False)

# Create pertenciles for all projects

## SCEA projects

First we get all SCEA projects from our ontology that have cellular type

In [53]:
server_name = 'http://194.4.103.244:3030'
service_name = 'ds'
request_url = server_name + '/' + service_name

query = '''
    PREFIX a: <http://www.semanticweb.org/alicia/ontologies/2020/8/singleCellRepositories#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT 
        ?projectID
    WHERE
    {
      ?project rdf:type a:Project ;
               a:PR.hasProjectID ?projectID ;
               a:SPR.isPartOfRepository "SingleCellExpresionAtlas" ;
               a:SPR.hasCellType ?cellType .                                               
    }

    GROUP BY ?projectID
'''

response = requests.post(request_url,
   data={'query': query})

projects_IDs = []

for project in response.json()['results']['bindings']:
    project_ID = project['projectID']['value']
    
    projects_IDs.append(project_ID)

len(projects_IDs)

99

Now that we have all the project IDs, we can get the percentile of each one.

In [54]:
n_projects = len(projects_IDs)

for n, project_ID in enumerate(projects_IDs):
    print(f"{n+1}/{n_projects}")
    time.sleep(1)
    
    groups, gen_names = get_percentiles_from_project(project_ID)

    if groups is None:
        clear_output(wait=True)
        continue
    
    dic = {}
    for d in [{'gen_name': gen_names}] + [{group['name']: group['percentiles']} for group in groups]:
        dic.update(d)
    
    df = pd.DataFrame(dic)
    df.to_csv(f'../SingleCell-Files/percentiles/{project_ID}.percentiles.csv', index=False)
    
    clear_output(wait=True)

99/99


# Creating index

## Example with 3 projects

First, we define the schema of the index. Each document will have a title and a content (project genes)

In [84]:
from whoosh.index import create_in
from whoosh.fields import *

schema = Schema(title=TEXT(stored=True), content=TEXT(stored=True))

Now, we will create the index in a directory

In [85]:
import os.path
import shutil

path = "../SingleCell-Files/index"

if not os.path.exists(path):
    os.mkdir(path)
else:
    shutil.rmtree(path)
    os.mkdir(path)

ix = create_in(path, schema)

In [86]:
writer = ix.writer()

We will test the index with 3 projects. 2 of them are studies of blood (E-MTAB-6386 and E-GEOD-139324). So they are suppose to have genes in common.

In [87]:
df0 = pd.read_csv('../SingleCell-Files/percentiles/E-GEOD-100911.percentiles.csv')
df1 = pd.read_csv('../SingleCell-Files/percentiles/E-MTAB-6386.percentiles.csv')
df2 = pd.read_csv('../SingleCell-Files/percentiles/E-GEOD-139324.percentiles.csv')

In [89]:
df0_genes = df0['gen_name'].tolist()
df1_genes = df1['gen_name'].tolist()
df2_genes = df2['gen_name'].tolist()

We can get the common gens between the projects, and some genes that are exclusive of each project. We will use these genes to test the index.

In [97]:
print(set(df0_genes).intersection(set(df1_genes)).intersection(set(df2_genes)))
print(list(set(df1_genes).intersection(set(df2_genes)))[:3])
print(df0_genes[:3])
print(list(set(df1_genes).difference(set(df2_genes)))[:3])
print(list(set(df2_genes).difference(set(df1_genes)))[:3])

set()
['ENSG00000160695', 'ENSG00000162222', 'ENSG00000134864']
['ENSDARG00000000001', 'ENSDARG00000000018', 'ENSDARG00000000019']
['ENSG00000287846', 'ENSG00000284732', 'ENSG00000130528']
['ENSG00000204572', 'ENSG00000181544', 'ENSG00000164185']


We add the three projects to the index

In [91]:
writer.add_document(title="E-GEOD-100911", content=' '.join(df0_genes))
writer.add_document(title="E-MTAB-6386", content=' '.join(df1_genes))
writer.add_document(title="E-GEOD-139324", content=' '.join(df2_genes))

writer.commit()

Now we can do some test queries with the genes saw before.

In [102]:
from whoosh.qparser import QueryParser

print("Searching for gene ENSDARG00000000001")

qp = QueryParser("content", ix.schema)
q = qp.parse(u"ENSDARG00000000001")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])
print()
print("Searching for gene ENSG00000160695")        

q = qp.parse(u"ENSG00000160695")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])
print()
print("Searching for gene ENSG00000287846")        

q = qp.parse(u"ENSG00000287846")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])

print()        
print("Searching for gene ENSG00000204572")        

q = qp.parse(u"ENSG00000204572")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])
        
print()
print("Searching for projects that contains genes starting with ENS")           

q = qp.parse(u"ENS*")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])

Searching for gene ENSDARG00000034326
CPU times: user 285 µs, sys: 18 µs, total: 303 µs
Wall time: 298 µs
E-GEOD-100911

Searching for gene ENSG00000160695
CPU times: user 288 µs, sys: 0 ns, total: 288 µs
Wall time: 291 µs
E-MTAB-6386
E-GEOD-139324

Searching for gene ENSG00000287846
CPU times: user 250 µs, sys: 15 µs, total: 265 µs
Wall time: 269 µs
E-MTAB-6386

Searching for gene ENSG00000204572
CPU times: user 269 µs, sys: 0 ns, total: 269 µs
Wall time: 273 µs
E-GEOD-139324

Searching for projects that contains genes starting with ENS
CPU times: user 3.47 s, sys: 54.5 ms, total: 3.53 s
Wall time: 3.53 s
E-GEOD-100911
E-MTAB-6386
E-GEOD-139324


## Index with all percentiles

In [1]:
from whoosh.index import create_in
from whoosh.fields import *

schema = Schema(title=TEXT(stored=True), content=TEXT(stored=True))

Now, we will create the index in a directory

In [2]:
import os.path
import shutil

path = "../SingleCell-Files/index"

if not os.path.exists(path):
    os.mkdir(path)
else:
    shutil.rmtree(path)
    os.mkdir(path)

ix = create_in(path, schema)

In [3]:
writer = ix.writer()

In [9]:
directory = '../SingleCell-Files/percentiles/'

for filename in os.listdir(directory):
    df = pd.read_csv(directory + filename)
    
    df_genes = df['gen_name'].tolist()
    df_cell_types = [x for x in df.columns if x != 'gen_name']
    content = df_genes + df_cell_types
    
    project_id = filename.split('/')[-1].split('.')[0]
    
    writer.add_document(title=project_id, content=' '.join(content))

writer.commit()

In [13]:
from whoosh.qparser import QueryParser

qp = QueryParser("content", ix.schema)
q = qp.parse(u"Neuron")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])

CPU times: user 277 µs, sys: 15 µs, total: 292 µs
Wall time: 296 µs
E-GEOD-100058
E-MTAB-7303
E-GEOD-75140


In [67]:
q = qp.parse(u"ENS*")

with ix.searcher() as s:
    %time results = s.search(q, limit=None)
    for result in results:
        print(result['title'])

CPU times: user 10 s, sys: 69.7 ms, total: 10.1 s
Wall time: 10.1 s
E-MTAB-7678
E-GEOD-89232
E-GEOD-106973
E-GEOD-70580
E-MTAB-6051
E-MTAB-6505
E-MTAB-7052
E-GEOD-75140
E-MTAB-6945
E-CURD-11
E-GEOD-75367
E-ENAD-13
E-GEOD-150728
E-CURD-6
E-GEOD-83139
E-MTAB-6524
E-GEOD-89910
E-MTAB-7098
E-GEOD-100426
E-MTAB-6379
E-MTAB-7051
E-MTAB-5802
E-MTAB-7094
E-MTAB-6925
E-MTAB-6308
E-ENAD-21
E-MTAB-4850
E-MTAB-5661
E-GEOD-98556
E-GEOD-124858
E-MTAB-5530
E-MTAB-6970
E-MTAB-6108
E-GEOD-110499
E-MTAB-7311
E-ENAD-14
E-MTAB-6487
E-MTAB-7660
E-MTAB-6386
E-MTAB-4888
E-GEOD-76312
E-MTAB-8810
E-MTAB-5485
E-GEOD-108221
E-GEOD-100911
E-MTAB-6058
E-GEOD-81682
E-MTAB-7008
E-MTAB-8077
E-MTAB-7703
E-GEOD-111727
E-MTAB-6879
E-MTAB-7149
E-GEOD-146122
E-MTAB-6362
E-MTAB-6385
E-MTAB-7606
E-GEOD-36552
E-GEOD-149689
E-MTAB-7303
E-MTAB-5727
E-GEOD-137537
E-GEOD-100618
E-GEOD-109979
E-MTAB-7249
E-MTAB-7381
E-MTAB-6987
E-GEOD-124472
E-MTAB-7037
E-GEOD-130473
E-GEOD-139324
E-MTAB-4547
E-MTAB-7901
E-MTAB-8559
E-GEOD-86618


#### Add cell types to index